# Channel  Quality Assessment, Pruning, and Motion Artifact Detection: Example Notebook

This notebook sketches how to prune bad channels and detect motion artefacts in fNIRS data

In [7]:
import cedalion
import cedalion.nirs
import cedalion.sigproc.quality as quality
import cedalion.xrutils as xrutils
import numpy as np
import xarray as xr
import pint
import matplotlib.pyplot as p
import scipy.signal
import os.path

### Loading raw CW-NIRS data from a SNIRF file and converting it to OD and CONC

This notebook uses a finger-tapping dataset in BIDS layout provided by Rob Luke. Download it [here](https://github.com/rob-luke/BIDS-NIRS-Tapping) and point the variable `DATADIR` to its location.

In [8]:
#DATADIR = "../../data/BIDS-NIRS-Tapping"
DATADIR = "C:/LocalDatasets/BIDS-NIRS-Tapping"

In [9]:
subjects  = [f"sub-{i:02d}" for i in [1,2,3,4,5]]
data={}

# choose and load subject snirf file
sbjnr = 0
DATAPATH = os.path.join(DATADIR, f"{subjects[sbjnr]}/nirs/{subjects[sbjnr]}_task-tapping_nirs.snirf")
element = cedalion.io.read_snirf(DATAPATH)

# convert to xarray data structure and convert to OD
# TBD: data = xrutils.snirf_to_xarray(element)
amp = element[0].data[0]
geo3d = element[0].geo3d
amp = amp.pint.dequantify().pint.quantify("volts") # work around missing units in snirf file
dpf = xr.DataArray([6, 6], dims="wavelength", coords={"wavelength" : amp.wavelength})
od = cedalion.nirs.int2od(amp)
conc = cedalion.nirs.beer_lambert(amp, geo3d, dpf)
data[subjects[sbjnr]] = xr.Dataset(
    data_vars = {
        "amp" : amp,
        "od"  : od,
        "conc": conc,
        "geo": geo3d
    })

dat = data["sub-01"]
dat

Magnitude,[[[0.0913686 0.0909875 0.0910225 ... 0.0941083 0.0940129 0.0944882] [0.1856806 0.186377 0.1836514 ... 0.1856486 0.1850836 0.1842172]] [[0.227516 0.2297024 0.2261366 ... 0.2264519 0.2271665 0.226713] [0.6354927 0.637668 0.6298023 ... 0.6072068 0.6087293 0.6091066]] [[0.1064704 0.1066212 0.1053444 ... 0.121114 0.1205022 0.1205441] [0.2755033 0.2761615 0.2727006 ... 0.2911952 0.2900544 0.2909847]] ... [[0.2027881 0.1996586 0.2004866 ... 0.2318743 0.2311941 0.2330808] [0.4666358 0.4554404 0.4561614 ... 0.4809749 0.4812827 0.4862896]] [[0.4885007 0.4802285 0.4818338 ... 0.6109142 0.6108118 0.613845] [0.8457658 0.825988 0.8259648 ... 0.975894 0.9756599 0.9826459]] [[0.6304559 0.6284427 0.6287045 ... 0.6810626 0.6809573 0.6818709] [1.2285622 1.2205907 1.2190002 ... 1.2729124 1.2727222 1.2755645]]]
Units,volt
Magnitude,[[[0.040420720851589244 0.04460046098449573 0.0442158667266921 ... 0.010876348689028297 0.01189058853204466 0.006847636450666034] [0.023820496835156947 0.020076986081598715 0.03480909472899116 ... 0.023992850639162802 0.027040875264083337 0.031732993935654014]] [[-0.008280063468686781 -0.01784405593180633 -0.002198739340328898 ... -0.003592058220542012 -0.006742726672860488 -0.0047443982578777594] [-0.03725579234490354 -0.04067296052603699 -0.028261149412724014 ... 0.008275386235838757 0.005771141488662178 0.0051515177576299375]] [[0.1005582275517943 0.09914287354277777 0.11119025710690507 ... -0.02830701101541003 -0.023242770227372977 -0.023590421283804962] [0.04993799639529682 0.047551763623795935 0.06016311306879892 ... -0.005456229532249399 -0.0015308884573803712 -0.004733085331725723]] ... [[0.09543410489184105 0.11098678842280141 0.10684828479073327 ... -0.038599718102985896 -0.03566192105523218 -0.04378947818067423] [0.03858010875753767 0.06286432773409747 0.06128249638988582 ... 0.008314104856099373 0.007674359302522161 -0.0026751405572218796]] [[0.15506580435158798 0.17214467649471782 0.16880746748060918 ... -0.06854981498765861 -0.06838218329243445 -0.07333574448986935] [0.10250044669095155 0.12616269025036977 0.1261907782199553 ... -0.040611038136814874 -0.040371126750177955 -0.04750589562552556]] [[0.0580532188103781 0.06125157285758024 0.06083507429037254 ... -0.019157798285875216 -0.019003174994086804 -0.020343916456372766] [0.024377017778147235 0.030886638430615728 0.03219054575814673 ... -0.01108594305570406 -0.010936510768482734 -0.013167265399560904]]]
Units,dimensionless
Magnitude,[[[0.13358239209978298 0.003830283928282594 0.3485663033081771 ... 0.44265804481800175 0.5026875842869235 0.6632956216585129] [-0.7839218209250455 -0.7640062849016894 -0.6372321468890523 ... 0.23076741243710222 0.20512673319586014 0.16981311715887415] [0.11212379127914074 0.0731324774375613 0.2338866759571778 ... 0.15960301220515644 0.19659631898706503 0.1288759014200041] ... [-0.08996858531937914 0.29760277238751043 0.3035425193755599 ... 0.570968554065664 0.5273568764191446 0.37714069302428926] [0.9042366397918423 1.340078204934004 1.381330424752029 ... -0.27197257740205283 -0.26748647744434617 -0.40128365125815096] [-0.18579734906426823 0.42721870170643006 0.6069404693046488 ... -0.30557630132353186 -0.29587728904779226 -0.4927704425869436]] [[0.4302890462699156 0.5291122886883187 0.39408017947069485 ... -0.03812950019946504 -0.048780953537845895 -0.16954954368367178] [0.19718416808530376 0.07475108502790183 0.21473067649191507 ... -0.13048136047941516 -0.15862865728380518 -0.12125825687699171] [1.1069744947570999 1.1055519748193772 1.1824229590870727 ... -0.38395393707435815 -0.34006765489373475 -0.31841423445257] ... [1.1310069656643422 1.1631091585413527 1.1132916692449673 ... -0.6597509742064699 -0.6094788888477872 -0.6460548286777947] [1.8250852563797992 1.8988537984724563 1.8366005072863651 ... -0.855162631152548 -0.8545173888400667 -0.8731183537487607] [3.671881628238195 3.6383236051532477 3.5444730339882793 ... -1.0728951396816504 -1.0669728023777476 -1.0756417750550116]]]
Units,micromolar
Magnitude,[[-0

### Channel Pruning

In [22]:
# initialize UnitRegistry
ureg = cedalion.units

# we can assess channel quality by the following metrics
snr_thresh = 20 # the SNR (std/mean) of a channel
amp_threshs = [0.1, 3]*ureg.volt # whether a channel's amplitude is within a certain range
sd_threshs = [1, 4.5]*ureg.cm # whether a channels distance is within a certain range


# the following functions assess the above criteria individually and return a measurement list (MeasList)
# that contains only channels that meet the criteria

# SNR thresholding
snr, MeasList_snr, drop_list_snr = quality.snr_range(dat.amp, snr_thresh)
print(f"The channels passing the snr threshold are: {MeasList_snr}")
print(f"The channels failing the snr threshold are: {drop_list_snr}")

# Source Detector Separation thresholding
ch_dist, MeasList_sd, drop_list_sd = quality.sd_range(dat.amp, dat.geo, sd_threshs)
print(f"The channels passing the SD threshold are: {MeasList_sd}")
print(f"The channels failing the SD threshold are: {drop_list_sd}")

# Amplitude thresholding
MeasList_amp, drop_list_amp = quality.amp_range(dat.amp, amp_threshs)
print(f"The channels passing the amplitude thresholds are: {MeasList_amp}")
print(f"The channels failing the amplitude thresholds are: {drop_list_amp}")

# To comprehensively prune channels from the dataset across criteria, we can use the quality.prune 
# function that incorporates all the above helper functions
data_pruned, drop_list = quality.prune(dat.amp, dat.geo, snr_thresh, amp_threshs, sd_threshs)
print(f"The channels passing all thresholds are: {data_pruned.coords['channel'].values}")
print(f"The channels failing any or all thresholds are: {drop_list}")


The channels passing the snr threshold are: ['S1D1', 'S1D2', 'S1D3', 'S1D9', 'S2D1', 'S2D3', 'S2D4', 'S2D10', 'S3D2', 'S3D3', 'S3D11', 'S4D3', 'S4D12', 'S5D5', 'S5D6', 'S5D13', 'S6D7', 'S6D14', 'S7D15', 'S8D7', 'S8D16']
The channels failing the snr threshold are: ['S4D4' 'S5D7' 'S6D5' 'S6D8' 'S7D6' 'S7D7' 'S8D8']
The channels passing the SD threshold are: ['S1D1', 'S1D2', 'S1D3', 'S2D1', 'S2D3', 'S2D4', 'S3D2', 'S3D3', 'S4D3', 'S4D4', 'S5D5', 'S5D6', 'S5D7', 'S6D5', 'S6D7', 'S6D8', 'S7D6', 'S7D7', 'S8D7', 'S8D8']
The channels failing the SD threshold are: ['S1D9' 'S2D10' 'S3D11' 'S4D12' 'S5D13' 'S6D14' 'S7D15' 'S8D16']
The channels passing the amplitude thresholds are: ['S1D1', 'S1D2', 'S1D3', 'S1D9', 'S2D1', 'S2D3', 'S2D4', 'S2D10', 'S3D2', 'S3D3', 'S3D11', 'S4D3', 'S4D4', 'S4D12', 'S5D5', 'S5D6', 'S5D7', 'S5D13', 'S6D5', 'S6D7', 'S6D8', 'S6D14', 'S7D6', 'S7D7', 'S7D15', 'S8D7', 'S8D8', 'S8D16']
The channels failing the amplitude thresholds are: []
The channels passing all thresholds 